<a href="https://colab.research.google.com/github/NandiniLReddy/Teaching_AgenticDesign101/blob/main/PromptChaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Chaining implementation

## background info

implementing prompt chaining requires direct to seqential function calls within the scripts; so some specialized frameworks like control, flow, state and component integration is requried:

so some frameworks like langchain, langgraph, crew ai, and google ADK provides such environment:

## langchain
so langchain and lang-graph are suitable choices as their core apis are explicitly designed for composing chains and graph operations.
## langgraph
langchain provides foundational abstractions for linear sequences,
while langgraph extends these capabilities to support stateful and cyclical computations which are necessary for implementing more sophisticated agentic behaviours

### this is the implementation of 2 step prompt chaining that functions as a data processing pipeline

## initial stage  --  parse unstrcutured text and extract specific information
## subsequent stage - receives this structured output and transforms into a structured data format

## start

In [7]:
pip install langchain langchain-community langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
#load llm
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [19]:
#prompt-1: Extract info
prompt_extract = ChatPromptTemplate.from_template("Trandform the following specifications from the following text: \n\n{text_input}")

In [20]:
#prompt-2: Transform to JSON
prompt_transform = ChatPromptTemplate.from_template("Transform the following text into a JSON object with 'cpu', 'memory', and 'storage' as keys: \n\n{specifications}")

In [21]:
#building chain using LCEL(Langchain expression language)
extraction_chain = prompt_extract | llm | StrOutputParser()
#the StrOutputParser() converts the LLM's message output into a simple string.

In [22]:
full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)
#the full chain passes the output of extraction chain into the 'specifications'

In [23]:
#let's run
input_text = "The new laptop model features a 3.5 Ghz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

In [24]:
final_result = full_chain.invoke({"text_input": input_text})

In [17]:
print(final_result)

{
    "cpu": "3.5 Ghz octa-core",
    "memory": "16GB",
    "storage": "1TB NVMe SSD"
}


# lang-graph looping mechanism:

In [25]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Define the State for graph
# This will be passed between nodes and updated.
class GraphState(TypedDict):
    """
    Represents the state of our graph, including initial input and processed text.

    Attributes:
        initial_input (str): The original text input.
        current_text (str): The text being processed and refined through iterations.
        iteration_count (int): Counter for loop iterations.
    """
    initial_input: str
    current_text: str
    iteration_count: int

# Define a prompt for refinement within the loop using the existing LLM
refinement_prompt = ChatPromptTemplate.from_template(
    "Given the following text, please summarize it more concisely or refine its key points. This is refinement iteration {iteration_count}.\n\nText: {input_text}"
)
refinement_chain = refinement_prompt | llm | StrOutputParser()

# Define the nodes (functions) for our graph
def start_node(state: GraphState) -> GraphState:
    #Initializes the state with the initial_input and sets iteration_count to 0.
    print("---> STARTING WORKFLOW <---")
    return {"current_text": state['initial_input'], "iteration_count": 0}

def loop_processing_node(state: GraphState) -> GraphState:
    # A node that processes the current_text using the LLM, increments a counter.
    print(f"---> LOOPING/PROCESSING (Iteration: {state['iteration_count'] + 1}) <---")
    current_count = state['iteration_count']
    new_count = current_count + 1

    text_to_refine = state['current_text']
    refined_text = refinement_chain.invoke({
        "input_text": text_to_refine,
        "iteration_count": new_count
    })
    print(f"   [Iteration {new_count}] Refined text snippet: {refined_text[:70]}...") # Print a snippet for progress
    return {"current_text": refined_text, "iteration_count": new_count}

def final_node(state: GraphState) -> GraphState:
    # The end process state
    print("---> FINALIZING WORKFLOW <---")
    # The final_node simply passes the last refined text along.
    return {"current_text": state['current_text']}

# function to decide the next step after the loop_processing_node
def decide_next_step(state: GraphState) -> str:
    max_iterations = 3 # Define how many times to loop for this example
    if state['iteration_count'] < max_iterations:
        print(f"---DECISION: LOOPING BACK (Current iteration: {state['iteration_count']} / {max_iterations})---")
        return "loop_processing"
    else:
        print(f"---DECISION: PROCEEDING TO FINAL NODE (Iterations complete: {state['iteration_count']} / {max_iterations})---")
        return "final_step"


# Build the Langgraph graph
workflow = StateGraph(GraphState)

# Add the nodes
workflow.add_node("start_step", start_node)
workflow.add_node("loop_processing", loop_processing_node)
workflow.add_node("final_step", final_node)

# Set the entry point and edges
workflow.add_edge(START, "start_step")
workflow.add_edge("start_step", "loop_processing")

# Add conditional edge for the loop
workflow.add_conditional_edges(
    "loop_processing", # From the 'loop_processing' node
    decide_next_step, # Use the decision function
    {
        "loop_processing": "loop_processing", # If 'loop_processing', loop back to itself
        "final_step": "final_step" # If 'final_step', go to the final node
    }
)

workflow.add_edge("final_step", END)

# Compile the graph
app = workflow.compile()

print("Langgraph workflow compiled successfully with an LLM-powered loop example.")

Langgraph workflow compiled successfully with an LLM-powered loop example.


In [26]:
# Invoke the Langgraph app with a concrete input text
initial_text_for_langgraph = (
    "The quick brown fox jumps over the lazy dog. This sentence is a classic example "
    "used for testing typewriters and computer keyboards. It contains all the letters "
    "of the English alphabet. We want to refine and summarize this text multiple times "
    "to demonstrate a Langgraph loop with an LLM, making it progressively more concise."
)

print(f"\nInitial input for Langgraph: {initial_text_for_langgraph[:100]}...")

final_langgraph_state = app.invoke({
    "initial_input": initial_text_for_langgraph,
    "current_text": "", # 'current_text' will be initialized by start_node with 'initial_input'
    "iteration_count": 0 # 'iteration_count' will be initialized by start_node
})

# Print the final processed text from the state
print("\nLanggraph Output (Final Processed Text after iterations):")
print(final_langgraph_state['current_text'])


Initial input for Langgraph: The quick brown fox jumps over the lazy dog. This sentence is a classic example used for testing typ...
---> STARTING WORKFLOW <---
---> LOOPING/PROCESSING (Iteration: 1) <---
   [Iteration 1] Refined text snippet: The sentence "The quick brown fox jumps over the lazy dog" is commonly...
---DECISION: LOOPING BACK (Current iteration: 1 / 3)---
---> LOOPING/PROCESSING (Iteration: 2) <---
   [Iteration 2] Refined text snippet: "The quick brown fox jumps over the lazy dog" is a sentence used to te...
---DECISION: LOOPING BACK (Current iteration: 2 / 3)---
---> LOOPING/PROCESSING (Iteration: 3) <---
   [Iteration 3] Refined text snippet: "The quick brown fox jumps over the lazy dog" is a sentence used to te...
---DECISION: PROCEEDING TO FINAL NODE (Iterations complete: 3 / 3)---
---> FINALIZING WORKFLOW <---

Langgraph Output (Final Processed Text after iterations):
"The quick brown fox jumps over the lazy dog" is a sentence used to test typewriters and keyboar